In [243]:
import pandas as pd

In [244]:
file_path_episodes = "../exploration/BIG_DF_ML.ipynb"

db01 = pd.read_csv("../gitignore/title_basics_traite.csv")
db02 = pd.read_csv("../gitignore/title_ratings_final.tsv", sep="\t")
db03 = pd.read_csv("../gitignore/title.akas_final.tsv", sep="\t")
db04 = pd.read_csv("../gitignore/tmdb_ml_final.csv")
db05 = pd.read_csv("../gitignore/data_bechdel.csv")
db07 = pd.read_csv("../gitignore/name.basics.tsv", sep="\t") 
db08 = pd.read_csv("../gitignore/title.crew.tsv", sep="\t") 

In [245]:
dbmerge_1 = pd.merge(db01, db02, right_on='title_ratings_tconst', left_on='tconst', how='left') #Title Basics + Title Ratings
dbmerge_2 = pd.merge(dbmerge_1, db03, left_on='tconst', right_on='titleId', how='left') # + Title Akas
dbmerge_3 = pd.merge(dbmerge_2, db04, left_on='tconst', right_on='tmdb_imdb_id', how='left') # + TMDB Full
dbmerge_4 = pd.merge(dbmerge_3, db05, left_on='tconst', right_on='imdbid', how='left') # + Bechdel
dbmerge_4 = pd.merge(dbmerge_4, db08, left_on='tconst', right_on='tconst', how='left') # + Title Crew
dbmerge_4 = pd.merge(dbmerge_4, db07, left_on='directors', right_on='nconst', how='left') # + Name Basics

In [185]:
#dbmerge_4.to_csv("../gitignore/BIG_DF_ML.csv", index=False)

# Nouveau test avec votants > 327 / année de sortie > 1920 / 5 films pays sélectionné + 5 films autres pays

In [246]:
BIG_DF_ML = dbmerge_4.drop(columns=[
    'titleType',
    'genres', 
    'decade', 
    'Adult',
    'Short',
    'movie',
    'tmdb_Comedy',
    'tmdb_Adventure',
    'tmdb_Drama',
    'tmdb_Crime',
    'tmdb_Action',
    'tmdb_Documentary',
    'tmdb_Animation',
    'tmdb_Mystery',
    'tmdb_Horror',
    'tmdb_Western',
    'tmdb_Science Fiction',
    'tmdb_Thriller',
    'tmdb_Romance',
    'tmdb_Fantasy',
    'tmdb_Family',
    'tmdb_History',
    'tmdb_Music',
    'tmdb_War', 
    'ordering',
    'region',
    'language',
    'types',
    'attributes',
    'isOriginalTitle',
    'birthYear',
    'deathYear',
    'primaryProfession',
    'knownForTitles',
    'directors',
    'writers'
])

In [247]:
BIG_DF_ML.loc[(dbmerge_4['title_ratings_averageRating'].isnull())&(dbmerge_4['tmdb_vote_average'].isnull()), ['title_ratings_tconst', 'title_ratings_averageRating', 'tmdb_vote_average']]

,title_ratings_tconst,title_ratings_averageRating,tmdb_vote_average
9,NaN,NaN,NaN
10,NaN,NaN,NaN
11,NaN,NaN,NaN
12,NaN,NaN,NaN
13,NaN,NaN,NaN
...,...,...,...
688329,NaN,NaN,NaN
688335,NaN,NaN,NaN
688336,NaN,NaN,NaN
688337,NaN,NaN,NaN


In [248]:
BIG_DF_ML3 = BIG_DF_ML.dropna(subset=['title_ratings_averageRating','tmdb_vote_average'],how='all')

In [249]:
def moyenne_ponderee(ligne):

    # Si 'title_ratings_averageRating' est NaN, on ne prend que 'tmdb_vote_average', et vice versa

    if pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        return ligne['tmdb_vote_average']  # Si title_ratings_averageRating est vide, prendre tmdb_vote_average

    elif pd.isna(ligne['tmdb_vote_average']) and not pd.isna(ligne['title_ratings_averageRating']):
        return ligne['title_ratings_averageRating']  # Si tmdb_vote_average est vide, prendre title_ratings_averageRating

    elif not pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        # Si les deux colonnes ont des valeurs, calculer la moyenne pondérée
        return (ligne['title_ratings_averageRating'] * ligne['title_ratings_numVotes'] + ligne['tmdb_vote_average'] * ligne['tmdb_vote_count']) / (ligne['title_ratings_numVotes'] + ligne['tmdb_vote_count'])  # Moyenne simple, à ajuster si besoin !
    else:
        return np.nan  # Si les deux sont NaN, retourner NaN


BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)

C:\Users\frale\AppData\Local\Temp\ipykernel_12236\2696331554.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)


In [250]:
# Suppression des films sortis avant 1920
BIG_DF_ML3 = BIG_DF_ML3[BIG_DF_ML3['startYear'] >= 1920]

In [251]:
BIG_DF_ML3.describe()

,startYear,runtimeMinutes,title_ratings_averageRating,title_ratings_numVotes,tmdb_popularity,tmdb_release_date,tmdb_runtime,tmdb_vote_average,tmdb_vote_count,rating,notes
count,327847.000000,327847.00000,313459.000000,3.134590e+05,201312.000000,201312.000000,201312.000000,201312.000000,201312.000000,9586.000000,327847.000000
mean,1996.062666,84.33487,6.177591,3.704412e+03,3.382595,1968.703589,86.042993,4.377985,95.716232,2.157730,5.949025
std,26.018551,165.19044,1.385250,3.685595e+04,26.082977,225.222552,109.149827,2.812918,726.228894,1.074264,1.782200
min,1920.000000,0.00000,1.000000,5.000000e+00,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1980.000000,74.00000,5.300000,1.900000e+01,0.695000,1975.000000,78.000000,1.761500,1.000000,1.000000,5.200000
50%,2007.000000,90.00000,6.300000,6.300000e+01,1.305000,2004.000000,90.000000,5.300000,3.000000,3.000000,6.200000
75%,2016.000000,101.00000,7.100000,3.270000e+02,2.512000,2015.000000,103.000000,6.417000,13.000000,3.000000,7.090908
max,2026.000000,59460.00000,10.000000,2.965080e+06,5089.969000,2024.000000,43200.000000,10.000000,33630.000000,3.000000,10.000000


Beaucoup de films (environ 3000) ont des durées égales à 0. Après vérification, ce sont juste des films dont la durée n'est pas renseignée sur IMDB. Nous avons décidé des les garder car certains sont bien notés.

In [252]:
# Suppression de 75% des films en se basant sur le nombre de votes Q3
BIG_DF_ML3 = BIG_DF_ML3[BIG_DF_ML3['title_ratings_numVotes'] >= 327]

In [253]:
BIG_DF_ML4 = BIG_DF_ML3.drop(['title_ratings_averageRating','tmdb_vote_average','title_ratings_tconst','titleId','tmdb_imdb_id','imdbid', 'primaryName'], axis=1)

In [254]:
def runtimeMinutes(ligne):

    # Si 'runtimeMinutes' est 0, il prend la valeur 'tmdb_runtime'
    BIG_DF_ML4.loc[(BIG_DF_ML4['runtimeMinutes'] == 0)&(BIG_DF_ML4['tmdb_runtime'] != 0), 'runtimeMinutes'] = BIG_DF_ML4['tmdb_runtime']

runtimeMinutes(BIG_DF_ML4)

In [255]:
def runtimeMinutes(ligne):

    # Si 'tmdb_runtime' est 0, il prend la valeur 'runtimeMinutes'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_runtime'] == 0)&(BIG_DF_ML4['runtimeMinutes'] != 0), 'tmdb_runtime'] = BIG_DF_ML4['runtimeMinutes']

runtimeMinutes(BIG_DF_ML4)

In [256]:
BIG_DF_ML5 = BIG_DF_ML4.drop(columns =['tmdb_runtime','tmdb_release_date','tmdb_vote_count','tmdb_TV Movie'])

In [257]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['runtimeMinutes'])

In [258]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['startYear'])

In [259]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['title'])

In [260]:
# Pour notre ML, on remplace toutes les valeurs nulles des pays de production par False 
Remplacer = [
    'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
    'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE', 'tmdb_SU',
    'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR', 'tmdb_RU', 'tmdb_DK',
    'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC',
    'tmdb_FI', 'tmdb_NO', 'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
]
BIG_DF_ML5[Remplacer] = BIG_DF_ML5[Remplacer].fillna(False)

In [261]:
BIG_DF_ML5['rating'] = BIG_DF_ML5['rating'].fillna(0)

In [262]:
BIG_DF_ML5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78113 entries, 4344 to 688332
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   tconst                  78113 non-null  object 
 1   startYear               78113 non-null  int64  
 2   runtimeMinutes          78113 non-null  float64
 3   Action                  78113 non-null  bool   
 4   Adventure               78113 non-null  bool   
 5   Animation               78113 non-null  bool   
 6   Biography               78113 non-null  bool   
 7   Comedy                  78113 non-null  bool   
 8   Crime                   78113 non-null  bool   
 9   Documentary             78113 non-null  bool   
 10  Drama                   78113 non-null  bool   
 11  Family                  78113 non-null  bool   
 12  Fantasy                 78113 non-null  bool   
 13  Game-Show               78113 non-null  bool   
 14  History                 78113 non-null 

In [263]:
BIG_DF_ML5['tmdb_popularity'] = BIG_DF_ML5['tmdb_popularity'].fillna(0)

In [264]:
BIG_DF_ML5['nconst'] = BIG_DF_ML5['nconst'].fillna("")

In [265]:
BIG_DF_ML5[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']] = BIG_DF_ML5[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']].astype(bool)

In [266]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import seaborn as sns
import matplotlib.pyplot as plt

In [267]:
export = "../machine learning/DF_ML.csv.gz"
BIG_DF_ML5.to_csv(export, sep=",", index=False, compression="gzip")

In [268]:
df_ml = BIG_DF_ML5

# Test de ML avec cible sur les notes > 7

In [341]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78113 entries, 4344 to 688332
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   tconst                  78113 non-null  object 
 1   startYear               78113 non-null  int64  
 2   runtimeMinutes          78113 non-null  float64
 3   Action                  78113 non-null  bool   
 4   Adventure               78113 non-null  bool   
 5   Animation               78113 non-null  bool   
 6   Biography               78113 non-null  bool   
 7   Comedy                  78113 non-null  bool   
 8   Crime                   78113 non-null  bool   
 9   Documentary             78113 non-null  bool   
 10  Drama                   78113 non-null  bool   
 11  Family                  78113 non-null  bool   
 12  Fantasy                 78113 non-null  bool   
 13  Game-Show               78113 non-null  bool   
 14  History                 78113 non-null 

On veut entrainer notre modèle sur tout le dataframe et afficher UNIQUEMENT les k films les plus proches dont les notes sont supérieures à 7.

Première reco : 5 films avec genres similaires et du même pays de production 

In [538]:
def recommandation_finale(tconst):

    def recommandation(tconst):
        import pandas as pd
        from sklearn.neighbors import NearestNeighbors
        from sklearn.preprocessing import MinMaxScaler

        # Chargement des données
        df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

        # On récupère les valeurs genre et pays qui correspondent au film sélectionné
        df_selection = df_ml[df_ml['tconst'] == tconst]
        colonnes_genre = [
            'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
            'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'
        ]
        colonnes_pays = [
            'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
            'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
            'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
            'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
            'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
            'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
        ]

        genre = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_genre]
        pays = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_pays]

        index = df_ml.index
        df_ml_num = df_ml.select_dtypes('number')
        df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

        # Normalisation des colonnes numériques
        SN = MinMaxScaler()
        df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

        df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

        # On sépare notre df en deux groupes, en fonction de la note
        bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]
        
        # On crée une liste des colonnes à utiliser pour le modèle
        caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating', 
            'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
            'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 
            'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
            'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
            'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
            'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
            'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
            'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

        # On veut que nos recommandations aient automatiquement un genre en commun et un pays de prod en commun
        bons_films = bons_films[bons_films[genre].any(axis=1)]
        bons_films = bons_films[bons_films[pays].any(axis=1)]

        # On crée notre modèle
        model = NearestNeighbors(n_neighbors=10, metric='euclidean')
        model.fit(bons_films[caracteristiques])

        # On déclare les caractéristiques du film sélectionné par l'utilisateur
        caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst][caracteristiques]

        # On calcule les distances et indices des voisins
        distances, indices = model.kneighbors(caract_film)

        # On affiche la sélection des films en fonction des indices trouvés par le modèle
        if caract_film['notes'].values[0] > 0.7:
            distances = distances[0][1:6]
            indices = indices[0][1:6]
            selection = bons_films.iloc[indices]['tconst']
        else:
            distances = distances[0][0:5]
            indices = indices[0][0:5]
            selection = bons_films.iloc[indices]['tconst']

        selection = pd.DataFrame(selection).reset_index(drop=True)

        return selection

    def recommandation2(tconst):

        df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

        #On récupère les valeurs genre et pays qui correspondent au film selectionné
        df_selection = df_ml[df_ml['tconst'] == tconst]
        colonnes_genre = ['Action', 'Adventure',
        'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
        'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
        'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
        'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

        colonnes_pays = ['tmdb_US',
            'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
            'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
            'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
            'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
            'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
            'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA']

        genre = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_genre]
        pays = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_pays]

        index = df_ml.index
        df_ml_num = df_ml.select_dtypes('number')
        df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

        from sklearn.preprocessing import MinMaxScaler
        SN = MinMaxScaler()
        df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

        df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

        #On crée une liste des colonnes à utiliser pour le modèle
        caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating',
            'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
            'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
            'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
            'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 
            'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
            'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
            'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
            'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
            'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
            'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

        #On sépare notre df en deux groupes, en fonction de la note
        bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]

        #On veut que nos recommandations aient automatiquement un genre en commun et un pays de prod en commun
        bons_films = bons_films[bons_films[genre].any(axis=1)]
        bons_films = bons_films[~bons_films[pays].any(axis=1)]

        #On crée notre modèle
        model = NearestNeighbors(n_neighbors=10, metric='euclidean')
        model.fit(bons_films[caracteristiques])

        #On déclare les caractéristiques du film sélectionné par l'utilisateur
        caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst]
        caract_film = caract_film[caracteristiques]
        caract_film

        distances, indices = model.kneighbors(caract_film)

        #On affiche la selection des films en fonction des indices trouvés par le modèle
        if caract_film['notes'].values[0] > 0.7:
            distances = distances[0][1:6]
            indices = indices[0][1:6]
            selection = bons_films.iloc[indices]['tconst']
        else:
            distances = distances[0][0:5]
            indices = indices[0][0:5]
            selection = bons_films.iloc[indices]['tconst']

        selection = pd.DataFrame(selection).reset_index(drop=True)

        return selection2

    return afficher_resultats_similarite(selection, selection2)



In [539]:
recommandation_finale('tt0275209')

(      tconst
 0  tt0184512
 1  tt0342512
 2  tt0049746
 3  tt0051390
 4  tt0358987,
       tconst
 0  tt0102571
 1  tt0380229
 2  tt0102141
 3  tt0254942
 4  tt0105137)

In [533]:
selection = recommandation('tt0275209')
selection

,tconst
0,tt0184512
1,tt0342512
2,tt0049746
3,tt0051390
4,tt0358987


Deuxième reco : 5 films avec genres similaires mais d'un autre pays de production 

In [536]:
def recommandation2(tconst):

     df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

     #On récupère les valeurs genre et pays qui correspondent au film selectionné
     df_selection = df_ml[df_ml['tconst'] == tconst]
     colonnes_genre = ['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

     colonnes_pays = ['tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA']

     genre = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_genre]
     pays = [colonne for colonne in df_selection.columns if df_selection[colonne].iloc[0] == True and colonne in colonnes_pays]

     index = df_ml.index
     df_ml_num = df_ml.select_dtypes('number')
     df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

     df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating',
        'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
        'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
        'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
        'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 
        'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
        'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
        'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
        'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
        'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
        'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]

     #On veut que nos recommandations aient automatiquement un genre en commun et un pays de prod en commun
     bons_films = bons_films[bons_films[genre].any(axis=1)]
     bons_films = bons_films[~bons_films[pays].any(axis=1)]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=10, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     if caract_film['notes'].values[0] > 0.7:
          distances = distances[0][1:6]
          indices = indices[0][1:6]
          selection = bons_films.iloc[indices]['tconst']
     else:
          distances = distances[0][0:5]
          indices = indices[0][0:5]
          selection = bons_films.iloc[indices]['tconst']

     selection = pd.DataFrame(selection).reset_index(drop=True)

     return selection



In [537]:
selection2 = recommandation2('tt0099785')
selection2

,tconst
0,tt0102571
1,tt0380229
2,tt0102141
3,tt0254942
4,tt0105137


In [540]:
def recommendation_finale(tconst):
    import pandas as pd
    from sklearn.neighbors import NearestNeighbors
    from sklearn.preprocessing import MinMaxScaler

    # ===============================
    # Chargement des données
    # ===============================
    df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

    # ===============================
    # Définition des colonnes pour les genres et les pays
    # ===============================
    colonnes_genre = [
        'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
        'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History',
        'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
        'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'
    ]
    colonnes_pays = [
        'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN',
        'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR',
        'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN',
        'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT',
        'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO',
        'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
    ]

    # ===============================
    # Extraction des genres et pays du film sélectionné
    # ===============================
    df_selection = df_ml[df_ml['tconst'] == tconst]
    genre = [col for col in df_selection.columns if col in colonnes_genre and df_selection[col].iloc[0] == True]
    pays = [col for col in df_selection.columns if col in colonnes_pays and df_selection[col].iloc[0] == True]

    # ===============================
    # Normalisation des colonnes numériques
    # ===============================
    df_ml_num = df_ml.select_dtypes('number')  # Colonnes numériques
    df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])  # Colonnes non numériques
    scaler = MinMaxScaler()
    df_ml_num_SN = pd.DataFrame(scaler.fit_transform(df_ml_num), columns=df_ml_num.columns, index=df_ml.index)
    df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)  # Fusion des colonnes normalisées et catégoriques

    # ===============================
    # Sélection des caractéristiques pour le modèle
    # ===============================
    drop_columns = ['tconst', 'nconst', 'title', 'title_ratings_numVotes', 'rating'] + colonnes_genre + colonnes_pays
    caracteristiques = df_ml_encoded.columns.drop(drop_columns)

    # ===============================
    # Filtrage des films avec une note supérieure ou égale à 0,7
    # ===============================
    bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]

    # ===============================
    # Création des deux sous-ensembles
    # ===============================
    # Films partageant le même genre et pays
    df_genre_country = bons_films[bons_films[genre].any(axis=1) & bons_films[pays].any(axis=1)]

    # Films partageant le même genre mais excluant les pays
    df_genre_no_country = bons_films[bons_films[genre].any(axis=1) & ~bons_films[pays].any(axis=1)]

    # ===============================
    # Fonction de recommandation avec Nearest Neighbors
    # ===============================
    def get_recommendations(data, tconst_features):
        model = NearestNeighbors(n_neighbors=10, metric='euclidean')
        model.fit(data[caracteristiques])  # Entraînement du modèle
        distances, indices = model.kneighbors(tconst_features)  # Recherche des voisins
        return distances, indices

    # ===============================
    # Extraction des caractéristiques du film sélectionné
    # ===============================
    caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst][caracteristiques]

    # ===============================
    # Recommandations pour les films avec genres et pays
    # ===============================
    distances1, indices1 = get_recommendations(df_genre_country, caract_film)
    selection1 = (
        df_genre_country.iloc[indices1[0][1:6]]['tconst'].reset_index(drop=True)
        if caract_film['notes'].values[0] > 0.7
        else df_genre_country.iloc[indices1[0][0:5]]['tconst'].reset_index(drop=True)
    )

    # ===============================
    # Recommandations pour les films avec genres mais sans pays
    # ===============================
    distances2, indices2 = get_recommendations(df_genre_no_country, caract_film)
    selection2 = (
        df_genre_no_country.iloc[indices2[0][1:6]]['tconst'].reset_index(drop=True)
        if caract_film['notes'].values[0] > 0.7
        else df_genre_no_country.iloc[indices2[0][0:5]]['tconst'].reset_index(drop=True)
    )

    # ===============================
    # Retour des deux DataFrames
    # ===============================
    return selection1, selection2


In [541]:
recommandation_finale('tt0275209')


(      tconst
 0  tt0184512
 1  tt0342512
 2  tt0049746
 3  tt0051390
 4  tt0358987,
       tconst
 0  tt0102571
 1  tt0380229
 2  tt0102141
 3  tt0254942
 4  tt0105137)